In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("downlaoded")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

downlaoded


In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 加载诗歌
path="/content/drive/MyDrive/准备分情感.xlsx"
df = pd.read_excel(path)
df["text_full"] = df["作品名（繁体）"] + "。" + df["诗歌内容（繁体）"]

print("reading done")


reading done


In [ ]:
#top 1 2 3
emotion_emb = {k: model.encode(v, convert_to_tensor=True) for k, v in emotions.items()}

weights = []
for poem in df["text_full"]:
    poem_emb = model.encode(poem, convert_to_tensor=True)
    sims = {emo: float(util.cos_sim(poem_emb, emb)) for emo, emb in emotion_emb.items()}
    # 归一化
    total = sum(sims.values())
    sims_norm = {emo: val / total for emo, val in sims.items()}
    weights.append(sims_norm)

df_emotion = pd.DataFrame(weights)
df = pd.concat([df, df_emotion], axis=1)

# 计算排名
def top_n_emotions(row, n=3):
    sorted_emos = row.sort_values(ascending=False)
    top_n = sorted_emos.index[:n]
    top_n_vals = sorted_emos.values[:n]
    return pd.Series({f"top{i+1}_emotion": emo for i, emo in enumerate(top_n)} |
                     {f"top{i+1}_weight": val for i, val in enumerate(top_n_vals)})

df_top = df[df_emotion.columns].apply(top_n_emotions, axis=1)
df = pd.concat([df, df_top], axis=1)

# 最终保存
df.to_excel("/content/drive/MyDrive/李白情感比例final.xlsx", index=False)


 "爱与友情": """
        表达诗人对朋友、知己的深情厚谊；
        偶有爱情题材，情感真挚；
        关键词：送别、情谊、知己、相思、红颜、桃花、汪伦；
        代表意象：桃花潭、水、酒、赠言；
        示例诗句：桃花潭水深千尺，不及汪伦送我情。
    """,
    "哲理与人生": """
        探讨生命意义、时间流逝与命运无常；
        融合佛道思想，体现李白后期的哲理反思；
        关键词：人生、天地、虚无、浮生、梦、轮回、天命；
        代表意象：明月、长夜、浮云、流水、杯中酒；
        示例诗句：人生得意须尽欢，莫使金樽空对月。
    """

In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 加载诗歌
path="/content/drive/MyDrive/准备分情感.xlsx"
df = pd.read_excel(path)
df["text_full"] = df["作品名（繁体）"] + "。" + df["诗歌内容（繁体）"]

print("reading done")

emotions = {
    "豪放与激昂": """
        核心：抒发壮志、抱负和英雄豪情，气势磅礴，激昂奋发；
        关键词：豪放、壮怀、慷慨、激昂、边塞、远志、奋起、仰天长啸、万里长风、奋发、豪情；
        代表意象：大漠、边关、长风、江海、高楼、青云、大鹏、战马、旌旗；
        示例诗句：黄河之水天上来，奔流到海不复回；仰天大笑出门去，我辈岂是蓬蒿人。
    """,

    "喜悦与欢乐": """
        核心：描写愉快、轻松、饮酒作乐或游山玩水的愉悦情绪；
        关键词：欢愉、畅饮、宴会、游乐、快意、歌舞、赏景、笑语、琴酒、月下、乐趣、游赏；
        代表意象：江河、山水、花园、桃花、酒杯、宴席、明月、春日、游船、芳草、春风；
        示例诗句：花间一壶酒，独酌无相亲；举杯邀明月，对影成三人。
    """,

    "哀怨与悲伤": """
        核心：因离别、怀才不遇、家国忧愁或人生坎坷而产生的悲伤与无奈；
        关键词：愁怨、失意、哀伤、牢骚、怀才不遇、离别、哀叹、叹息、郁结、惆怅；
        代表意象：明月、江水、荒野、故乡、边关、残花、秋风、雨夜、荒村；
        示例诗句：抽刀断水水更流，举杯消愁愁更愁；总为浮云能蔽日，长安不见使人愁。
    """,

    "思乡与怀古": """
        核心：怀念故乡或古人古迹，感慨历史与时光流逝；
        关键词：思乡、怀古、长安、故人、古迹、江山、历史、岁月、远别、念旧、怀念；
        代表意象：孤帆、长江、古塔、名胜、青山、故乡、荒城、古道、旧居、古桥；
        示例诗句：举头望明月，低头思故乡；孤帆远影碧空尽，唯见长江天际流。
    """,

    "孤寂与落寞": """
        核心：漂泊他乡、独处孤寂、身世坎坷、无人理解的孤独感；
        关键词：孤独、漂泊、落寞、寂寥、远行、独坐、幽居、清夜、寒林、孤影、形单影只、荒野、长夜、冷月、空舟、秋风、孤帆、寂静、幽窗；
        代表意象：山水、江河、夜月、独舟、青灯、寒江、敬亭山、荒村、孤林、远山、长街、冷泉、空屋、孤塔；
        示例诗句：独坐敬亭山，众鸟高飞尽；孤云独去闲，徘徊复何之；浮云一别后，流水十年间。
    """,

    "友情与知己": """
        核心：赞美友情与知己，表达赠别、送行或热情情谊；
        关键词：友情、知己、赠别、送行、豪饮、桃花潭、汪伦、手足、同心、契阔、厚谊、相送、倾耳、相伴；
        代表意象：桃花潭、酒、舟船、宴席、春日、宴会、长亭、折柳、江边、酒旗、流水、花径；
        示例诗句：桃花潭水深千尺，不及汪伦送我情；与君歌一曲，请君为我倾耳听。
    """
}
#top 1 2 3
emotion_emb = {k: model.encode(v, convert_to_tensor=True) for k, v in emotions.items()}

weights = []
for poem in df["text_full"]:
    poem_emb = model.encode(poem, convert_to_tensor=True)
    sims = {emo: float(util.cos_sim(poem_emb, emb)) for emo, emb in emotion_emb.items()}
    # 归一化
    total = sum(sims.values())
    sims_norm = {emo: val / total for emo, val in sims.items()}
    weights.append(sims_norm)

df_emotion = pd.DataFrame(weights)
df = pd.concat([df, df_emotion], axis=1)

# 计算排名
def top_n_emotions(row, n=3):
    sorted_emos = row.sort_values(ascending=False)
    top_n = sorted_emos.index[:n]
    top_n_vals = sorted_emos.values[:n]
    return pd.Series({f"top{i+1}_emotion": emo for i, emo in enumerate(top_n)} |
                     {f"top{i+1}_weight": val for i, val in enumerate(top_n_vals)})

df_top = df[df_emotion.columns].apply(top_n_emotions, axis=1)
df = pd.concat([df, df_top], axis=1)

# 最终保存
df.to_excel("/content/drive/MyDrive/李白情感比例3.0final.xlsx", index=False)



reading done


In [ ]:
!pip install opencc-python-reimplemented


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install opencc-python-reimplemented fuzzywuzzy python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.3 MB/s eta 0:00:00


In [ ]:
# -----------------------------
# 0️⃣ 安装依赖
# -----------------------------
!pip install sentence-transformers opencc-python-reimplemented


In [ ]:

# -----------------------------
# 1️⃣ 导入库
# -----------------------------
import pandas as pd
from opencc import OpenCC
from sentence_transformers import SentenceTransformer, util

cc = OpenCC('t2s')

# -----------------------------
# 2️⃣ 加载诗歌数据
# -----------------------------
# 假设 Excel 有列 '作品名' 和 '诗歌内容'
df = pd.read_excel("/content/drive/MyDrive/准备分情感.xlsx")
df["text_clean"] = df["诗歌内容（繁体）"].apply(lambda x: cc.convert(str(x)).replace(" ",""))

# -----------------------------
# 3️⃣ 定义情感及示例文本
# -----------------------------
emotions = {
    "豪放与激昂": "抒发壮志、抱负和英雄豪情，气势磅礴，激昂奋发，大漠边关，仰天长啸",
    "喜悦与欢乐": "描写愉快、轻松、饮酒作乐或游山玩水的愉悦情绪，宴会游乐，月下欢歌",
    "哀怨与悲伤": "离别、怀才不遇、家国忧愁或人生坎坷的悲伤与无奈，抽刀断水、举杯消愁",
    "思乡与怀古": "怀念故乡或古人古迹，感慨历史与时光流逝，孤帆远影，长江天际",
    "孤寂与落寞": "漂泊他乡、独处孤寂、身世坎坷，无人理解的孤独感，敬亭山独坐、孤舟寒江",
    "友情与知己": "友情、知己、赠别、送行，桃花潭水、汪伦相送，长亭折柳"
}

# -----------------------------
# 4️⃣ 构建情感向量
# -----------------------------
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

emotion_texts = list(emotions.values())
emotion_names = list(emotions.keys())
emotion_embeddings = model.encode(emotion_texts, convert_to_tensor=True)

# -----------------------------
# 5️⃣ 多标签匹配函数（top 3）
# -----------------------------
def match_emotions_topn(text, top_n=3):
    text_emb = model.encode(text, convert_to_tensor=True)
    sims = util.pytorch_cos_sim(text_emb, emotion_embeddings)[0]
    # 获取 top_n 索引
    top_indices = sims.argsort(descending=True)[:top_n]
    results = [(emotion_names[i], float(sims[i])) for i in top_indices]
    return results

# -----------------------------
# 6️⃣ 对每首诗进行匹配
# -----------------------------
df["emotion_top3"] = df["text_clean"].apply(lambda x: match_emotions_topn(x, top_n=3))

# -----------------------------
# 7️⃣ 保存结果
# -----------------------------
df.to_excel("/content/li_bai_emotion_top3.xlsx", index=False)
print("✅ 已生成多标签情感匹配结果")


✅ 已生成多标签情感匹配结果


In [ ]:
def match_emotions_top3_columns(text):
    top3 = match_emotions_topn(text, top_n=3)
    # 返回三列
    col1 = f"{top3[0][0]} ({top3[0][1]:.2f})" if len(top3) > 0 else ""
    col2 = f"{top3[1][0]} ({top3[1][1]:.2f})" if len(top3) > 1 else ""
    col3 = f"{top3[2][0]} ({top3[2][1]:.2f})" if len(top3) > 2 else ""
    return pd.Series([col1, col2, col3])

df[["emotion_1", "emotion_2", "emotion_3"]] = df["text_clean"].apply(match_emotions_top3_columns)

# -----------------------------
# 7️⃣ 保存结果
# -----------------------------
df.to_excel("/content/li_bai_emotion_top3分割_columns.xlsx", index=False)
print("✅ 已生成拆分为三列的多标签情感匹配结果")

✅ 已生成拆分为三列的多标签情感匹配结果


In [ ]:
# -----------------------------
# 6️⃣ 对每首诗进行匹配，并拆成三列名称 + 三列得分
# -----------------------------
def match_emotions_top3_name_score(text):
    top3 = match_emotions_topn(text, top_n=3)
    # 名称列
    name1 = top3[0][0] if len(top3) > 0 else ""
    name2 = top3[1][0] if len(top3) > 1 else ""
    name3 = top3[2][0] if len(top3) > 2 else ""
    # 得分列
    score1 = top3[0][1] if len(top3) > 0 else None
    score2 = top3[1][1] if len(top3) > 1 else None
    score3 = top3[2][1] if len(top3) > 2 else None
    return pd.Series([name1, name2, name3, score1, score2, score3])

df[["emotion_1_name","emotion_2_name","emotion_3_name",
    "emotion_1_score","emotion_2_score","emotion_3_score"]] = df["text_clean"].apply(match_emotions_top3_name_score)

# -----------------------------
# 7️⃣ 保存结果
# -----------------------------
df.to_excel("/content/li_bai_emotion_top3_name_score.xlsx", index=False)
print("✅ 已生成三列名称 + 三列得分的多标签情感匹配结果")
